In [1]:
using PyCall
@pyimport nltk

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2499: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [2]:
include("load_embeddings.jl")

load_embeddings (generic function with 1 method)

In [3]:
LL,word_index,indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL)

(50,268810)

[http://nlp.stanford.edu/pubs/SocherLinNgManning_ICML2011.pdf]

http://repository.cmu.edu/cgi/viewcontent.cgi?article=1054&context=robotics

http://en.wikipedia.org/wiki/Brown_Corpus#Part-of-speech_tags_used

In [4]:
function softmax(z)
    e.^z./sum(e.^z,1) 
end

softmax (generic function with 1 method)

In [5]:
LL |> typeof

Array{Float64,2}

In [6]:
type RNN
    L::Matrix{Float64}
    W_score::Matrix{Float64}
    W_label::Matrix{Float64}
    W::Matrix{Float64}
   
end


function RNN(L::Matrix{Float64}, num_labels::Int)
    input_width = 
    RNN(L,
        0.01*randn(1,input_width+1),
        0.01*randn(num_labels,input_width+1),
        0.01*randn(input_width,input_width*2+1))
end



function eval_embedding(rnn::RNN, input_A, input_B)
    bias_in = ones(1,size([input_A; input_B], 2))
    tanh(rnn.W*[input_A;input_B;bias_in])
end

function eval_score(rnn::RNN, pp)
    bias_in = ones(1,size(pp, 2))
    tanh(rnn.W_score*[pp;bias_in])
end

function eval_label(rnn::RNN, pp)
    bias_in = ones(1,size(pp, 2))
    softmax(rnn.W_label*[pp;bias_in])
end

eval_label (generic function with 1 method)

In [7]:
function word_adjancy_matrix(sentence_len::Int)
    [(ii==jj+1) || (ii==jj-1)  for ii in 1:sentence_len, jj in 1:sentence_len]
end



function eval_to_tree(rr::RNN, sentence::AbstractArray{String})
    backlookup = sentence
    ss = hcat([get_embedding(embeddings, token) for token in backlookup]...)
    AA = word_adjancy_matrix(size(ss,2))
    
    score_total = 0.0
    while(any(AA))
        iis, jjs = findn(AA)

        pps = eval_embedding(rr, ss[:,iis],ss[:,jjs])
        scores = eval_score(rr, pps)
        best_pair_ind = indmax(scores)
        
         
        ii_best, jj_best = sort([iis[best_pair_ind], jjs[best_pair_ind]])
        #The above makes a more readable output, but doesn't do anything useful,
        #it is same as below:
        #ii_best = iis[best_pair_ind]
        #jj_best = jjs[best_pair_ind]
        pp_best = pps[:,best_pair_ind]
        score_total+=scores[best_pair_ind]
        
        ss = [ss pp_best]
        backlookup = [backlookup, (backlookup[ii_best],backlookup[jj_best])]
        
        #Adjust Adjacency Matrix

        AA[ii_best,jj_best] = false
        AA[jj_best,ii_best] = false
        AA = [AA; AA[ii_best,:] | AA[jj_best,:]] # Add row
        AA = [AA AA[:,ii_best] | AA[:,jj_best]] # Add col
        AA[ii_best,:] = AA[jj_best,:] = false
        AA[:,ii_best] = AA[:,jj_best] = false #Remove anything that was adjacent to old

    end
    tree = backlookup[end]
    embedding = ss[:,end]
    tree, embedding, score_total
end
    

eval_to_tree (generic function with 1 method)

In [8]:
function tokenize(sentence::String)
    convert(Array{String,1},nltk.word_tokenize(sentence))
end

tokenize (generic function with 1 method)

In [11]:
sentence = "The house has a window."
sentence_toks = tokenize(sentence)
rr = RNN(LL,3)
eval_to_tree(rr,sentence_toks)




LoadError: input_width not defined
while loading In[11], in expression starting on line 3

In [ ]:
function bpts(rr::RNN, p)
    δp = 

In [12]:
function get_all_pos_tags()
    lines = split(open(readall,"brown_tags.txt"),"\n")
    desc_lines = filter(line -> contains(line,"||"), lines)
    tags = [split(line,"||")[1] for line in desc_lines]
end
pos_tags = get_all_pos_tags();

In [13]:
length(pos_tags)

226